# Laboratorio 9
Diego García 22404 <br>
César López 22535 <br>

github: https://github.com/DiegoGarV/Lab9-IA 

## Task 1 - Teoría

1. **Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models**
Los Modelos de Markov representan un sistema que cambia su estado con el tiempo. Por lo tanto, solo depende de su estado actual y no de como llegó a él. Por su parte, en los HMM los estados no se pueden observar de forma directa. Lo que si se puede observar, son resultados que dan pistas del estado real. Esto hace al HMM más realista en varios problemas. 

2. **Investigue qué son los factorial HMM (Hidden Markov Models)**
Un factorial HMM es una versión más avanzada del HMM. A diferencia del normal, este tiene varias cadenas ocultas de estados que funcionan al mismo tiempo y todas contribuyen en generar el resultado observable. Este es útil para encontrar los estados cuando dependen de multiples causas. Aunque hay varios estados ocultos, el resultado observable sigue siendo solo uno.

3. **Especifique en sus propias palabras el algoritmo Forward Backward para HMM**
Este es un algoritmo que permite calcular que tan probable es que se esté en cada uno de los estados ocultos en todo momento según todas las secuencias de observaciones. Para cada momento y estado se toma un tiempo pasado y se miran las probabilidades de cada estado futuro y se toma un tiempo futuro y se van calculando las probabilidades hacia atras.  

4. **En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)**
Porque el paso Forward no logra dar la probabilidad completa de haber estado en alguno de los estados, pues no indica las probabilidades futuras. Por ejemplo: yo quiero saber si la semana pasada mi novia estuvo feliz o triste. Uno de mis observables fue que el miercoles no sonrió. Ahí Forward me diría que lo más probable es que esté triste. Pero si además el jueves y el viernes estuvo sonriendo todo el día, Backward me mostraría que es más probable que esté feliz. Mi conclusión entonces podría ser que aunque el miercoles no sonrió, los demás días si así que lo más seguro es que haya estado feliz. Forward solo limitaría mi visión y me haría tener una conclusión imprecisa.

## Task 2 - Algoritmo Forward Backward en HMM